In [209]:
pwd

'/home/roger/src/python/liveproject/discovering-disease-outbreaks-base-master'

In [382]:
from unidecode import unidecode
from geonamescache import GeonamesCache
import numpy
import pandas as pd


gc = GeonamesCache()
cn = {}
city_names = []
states = []
state_names = []

cities = gc.get_cities()
city_id = list(cities.keys())[0]
#print(city_id)
#print(cities[city_id])

us_cities = [city for city in cities.values()
             if city['countrycode'] == 'US']
num_us_cities = len(us_cities)
#print(f"GeoNamesCache holds data for {num_us_cities} US cities.")

matched_cities_by_name = gc.get_cities_by_name('Rio de Janeiro')
#print(matched_cities_by_name)

states = gc.get_us_states()
cn = gc.get_countries_by_names()
countries = gc.get_countries()

matched_cities_list = gc.get_cities_by_name('San Francisco')
for i, san_francisco in enumerate(matched_cities_list):
    city_info = list(san_francisco.values())[0]
    country_code = city_info['countrycode']
    country = gc.get_countries()[country_code]['name']
   # print(f"The San Francisco at index {i} is located in {country}")

def readHeadlines(filename):

    headdata = []
    
    prob = open("prob.txt", 'w')
    with open(filename, 'r', encoding='utf8') as f:
        lnum = 0
        for line in f:
            lnum += 1
            cityname = ""
            countryname = ""
            statename = ""
            best = []
            
            while line.find("'s") >= 0:
                line = line.replace("'s", '')
                #print("----" + line)
            clean = ''.join(e for e in line.rstrip() if e.isalnum() or e == ' ')
            #print(clean)
            words = clean.split()
            
            matched_cities_list = getcity (words)
            #print("matched cities")
            #print(matched_cities_list)
            #print(" ")
            
            if len(matched_cities_list) > 0:
                best = matched_cities_list[0]
                if len(matched_cities_list) > 1:   
                    #print ("More than 1 city match")
                    #print (matched_cities_list)
                    best = max(matched_cities_list, key=lambda x: list(x.values())[0]['population'])
                    
                #print(best)
                #cityinfo = list(best.values())
                cityinfo = list(best.values())[0]
                #print (cityinfo)
                cityname = cityinfo['name']
                #print ("cityname " + cityname)
                lat = cityinfo['latitude']
                lon = cityinfo['longitude']
                country_code = cityinfo['countrycode']
                countryname = gc.get_countries()[country_code]['name']
                if country_code == 'US':
                    statename = cityinfo['admin1code']
            
            
            if len(cityname) == 0 and len(countryname) == 0 and len(statename) == 0:
                #print("write problem line")
                prob.write(line)
                continue
            
            headdata.append([line, countryname, cityname])
            #print(str(lnum) + '-> city=' + cityname + ' ' + 'statename=' + statename + ' ' + 'country=' + countryname)
            
    return headdata

def getcity(words):
    ret = ''
    prefix = ''
    mcl = ''
    
    if len(words) > 4:
        for i in range(len(words) - 3):
            wtmp = words[i] + ' ' + words[i+1] + ' ' + words[i+2] + ' ' + words[i+3]
            #print(wtmp)
            mcl = gc.get_cities_by_name(citymap(wtmp))
            if len(mcl) > 0:
                #print("1 getcity returning " + ret)
                return mcl
    if len(words) > 3:
        for i in range(len(words) - 2):
            wtmp = words[i] + ' ' + words[i+1] + ' ' + words[i+2]
            #print(wtmp)
            mcl = gc.get_cities_by_name(citymap(wtmp))
            if len(mcl) > 0:
                #print("2 getcity returning " + ret)
                return mcl
    if len(words) > 2:
        for i in range(len(words) - 1):
            wtmp = words[i] + ' ' + words[i+1]
            #print(wtmp)
            mcl = gc.get_cities_by_name(citymap(wtmp))
            #print(mcl)
            if len(mcl) > 0:
                #print("3 getcity returning " + ret)
                return mcl
    if len(words) > 0:
        for i in range(len(words)):
            w = words[i]
            mcl = gc.get_cities_by_name(citymap(w))
            if len(mcl) > 0:
                break
        
    #print("4 returning mcl")
    return mcl

def citymap(city):
    map = {"Sao Paulo": "São Paulo", "Geneve": "Genève", "St Louis": "St. Louis", "Cebu": "Cebu City",
           "Chicagos": "Chicago", "Bogota": "Bogotá", "Antigua": "Antigua Guatemala",
           "Rio De Janeiro": "Rio de Janeiro", "Brasilia": "Brasília", "Hillsborough": "Hillsboro",
           "Quebec": "Québec", "Cancun": "Cancún", "Mcallen": "McAllen", "Saint Johns": "St. Johns",
           "Arhus": "Aarhus", "Niteroi": "Niterói", "Joao Pessoa": "João Pessoa", "Mclean": "McLean",
           "Yaounde": "Yaoundé", "St Catharines": "St. Catharines", "Camacari": "Camaçari",
           "Montreal": "Montréal", "Druid Hills": "North Druid Hills", "Ribeirao Preto": "Ribeirão Preto",
           "Custodia": "Custódia", "Cantu": "Cantù", "Medellin": "Medellín", "Ciudad Acuña": "Ciudad Acuña",
           "Panama Citys": "Panama City", "Sao Jose": "São José"}
    
    #print("citymap " + city)
    
    if city in map.keys():
        #print("citymap returns " + map[city])
        return map[city]
    
    return city
    

def getcountry(words):
    print("getcountry")
    if len(words) > 2:
        for i in range(len(words) - 1):
            wtmp = words[i] + words[i+1]
            #print(wtmp)
            if cn.get(wtmp) != None:
                return wtmp
    if len(words) >= 1:
        for i in range(len(words)):
            w = words[i]
            #print(w)
            if cn.get(w) != None:
                return w
    return ""
            
#readHeadlines('probcopy.txt')                    
                    
headdata = readHeadlines('data/headlines.txt')

df = pd.DataFrame(headdata, columns=['Headline', 'Country', "City"])
print (df)


                

                                              Headline             Country  \
0                           Zika Outbreak Hits Miami\n       United States   
1                    Could Zika Reach New York City?\n       United States   
2                  First Case of Zika in Miami Beach\n       United States   
3            Mystery Virus Spreads in Recife, Brazil\n              Brazil   
4            Dallas man comes down with case of Zika\n       United States   
..                                                 ...                 ...   
605  Rumors about Rabies spreading in Jerusalem hav...              Israel   
606            More Zika patients reported in Indang\n         Philippines   
607  Suva authorities confirmed the spread of Rotav...                Fiji   
608       More Zika patients reported in Bella Vista\n  Dominican Republic   
609                   Zika Outbreak in Wichita Falls\n       United States   

              City  
0            Miami  
1    New York City  


In [371]:
import numpy
import pandas
